In [21]:
import pickle
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF
import cv2
from PIL import Image
import numpy as np
import torchvision.transforms as T
from Viz import UNet_Attention , load_checkpoint
from SegtUtils import Preprocessing , segmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
val_transforms = A.Compose(
    [
        A.Resize(height=256, width=512),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

In [23]:
model = UNet_Attention().to(DEVICE)

In [24]:
load_checkpoint(torch.load("./my_checkpoint.pth.tar"), model)

=> Loading checkpoint


In [25]:
drawing = False  # True if mouse is pressed
points = []  # List to store the points of the polygon
mask = None
frame = cv2.imread('./26.jpg')
frame = cv2.resize(frame , (512,256))
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
disp = frame.copy()

# Mouse callback function
def draw_polygon(event, x, y, flags, param):
    global drawing, points, mask, frame

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        points = [(x, y)]
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)  # Create a black mask image

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)
            points.append((x, y))

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)
        points.append((x, y))

        # Convert the points list to a NumPy array
        points_array = np.array(points)

        # Fill the polygon in the mask
        cv2.fillPoly(mask, [points_array], 255)

    cv2.imshow('image', frame)  # Display the updated frame

cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_polygon)

while True:
    cv2.imshow('image', frame)

    # Exit the loop when 'Esc' key is pressed
    if cv2.waitKey(1) == 27:
        break

# Save the mask as a white binary image
mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
img = cv2.bitwise_and(frame, mask)
cv2.imwrite('maxillomandab.jpg',img)
segt = segmentation(img,disp, model, val_transforms, DEVICE)
segt.show()

cv2.destroyAllWindows()